In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

In [2]:
import torch
print("当前设备：", torch.cuda.current_device())
print("设备名：", torch.cuda.get_device_name(0))

当前设备： 0
设备名： NVIDIA GeForce RTX 4090


In [3]:
from client import Client
from server_algo import Server_with_Algorithm
from server import Server
from datasets import load_dataset
import validation

# DASHSCOPE_API_KEY = os.getenv("DASHSCOPE_API_KEY")

In [4]:
# 1.首先选择要进行实验的client
clients = [Client(vectorstore_path="common_sense_db_0.6b")]
        #    Client(vectorstore_path="computer_science_coding_related_db_0.6b"),
        #    Client(vectorstore_path="law_related_db_0.6b"), Client(vectorstore_path="medicine_related_db_0.6b")]
# 2.对每个client进行加载
for c in clients:
    c.load_vectorstore()

/home/weiwei/RAGnet/client.py:63: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(model_name=model_path,


Vectorstore common_sense_db_0.6b loaded.


In [5]:
# import json
# client = Client(vectorstore_path="law_related_db_0.6b")
# client.load_vectorstore()
# output_path = "law_related.jsonl"
# with open(output_path, "w", encoding="utf-8") as f:
#     for doc in client.db.docstore._dict.values():
#         record = {
#             "page_content": doc.page_content,
#             # "metadata": doc.metadata  # 可选保留
#         }
#         f.write(json.dumps(record, ensure_ascii=False) + "\n")

# print(f"所有文档内容已保存至：{output_path}")

In [6]:
# 3.创建server对象，这里需要选择模型，请预先在ollama上部署
# server = Server(model_name="qwen3:4b")
server = Server(model_name="qwen3:4b")

/home/weiwei/RAGnet/server.py:21: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  self.llm = ChatOllama(model=model_name)


In [7]:
# 4.加载测试数据集
from datasets import Dataset

samples = Dataset.from_parquet("./test_dataset/web_questions/data/test-00000-of-00001.parquet")

import random
random_samples = samples.shuffle(seed=42).select(range(100))

In [8]:
# 5.进行评估，并生成csv文件
validation.evaluate_datasets(clients=clients, server=server, samples=random_samples,top_k=20,
                              output_csv="web_questions_result_baseline_topk_20.csv", dataset_name="web_questions")

Processing question: what was antoni gaudi inspired by?
Gold answers: ['William Morris', 'Eugène Viollet-le-Duc']
uncleaned answer: <think>
Okay, let's try to figure out what the user is asking here. They provided a bunch of context, mostly about architecture, design, and some historical references. The user's query is a bit vague, but looking at the last line of the context, it says "Today Up until the 20th century..." which might be a fragment. Maybe the user is asking about the history of fireplaces or mantels?

Looking at the context, there's a lot about gardens, mansions, architectural styles, and even some mentions of designers like Gary Anderson. There's also a part about the Château de Ferrières and other historical buildings. The user might be interested in how architecture has evolved over time, maybe comparing different styles or periods.

Another part mentions the 1960s and 1970s at USC, the influence of the Bauhaus, and the Watts Riots. That could be related to social and 